In [ ]:
from qdk import openqasm
import qsharp
from qsharp._simulation import NoiseConfig, run_qir_gpu, run_qir_cpu, run_qir_clifford
import qsharp_widgets

SHOTS = 100_000

# Initialize Q# with the adaptive profile configured
qsharp.init(target_profile=qsharp.TargetProfile.Adaptive_RIF)

# Compile a simplified version of the benzene sample to QIR.
qir = openqasm.compile("""
OPENQASM 3.0;
include "stdgates.inc";
qubit[4] q;
h q[0];
rz(1.86) q[0];
h q[0];
sdg q[0];
barrier q;
x q[1];
cx q[0], q[1];
cx q[0], q[2];
cx q[1], q[3];
h q;
bit[4] c = measure q;
""")

# Build a noise model
noise = NoiseConfig()
noise.h.loss = 0.005
noise.cx.loss = 0.005
noise.cx.x = 0.005

In [ ]:
# 1. Given a QIR string, we can simulate it using the GPU state-vector simulator.
#    We can also give it an optional noise model, and a seed for reproducibility.
results = run_qir_gpu(qir, shots=SHOTS, noise=noise)
bitstrings = [
    "".join("1" if bit == qsharp.Result.One else "0" for bit in measurement)
    for measurement in results
]
qsharp_widgets.Histogram(bitstrings, items = "top-25", sort="high-to-low")

In [ ]:
# 2. We can also simulate it using the CPU state-vector simulator,
#    which has the same API as the GPU simulator.
results = run_qir_cpu(qir, shots=SHOTS, noise=noise)
bitstrings = [
    "".join("1" if bit == qsharp.Result.One else "0" for bit in measurement)
    for measurement in results
]
qsharp_widgets.Histogram(bitstrings, items = "top-25", sort="high-to-low")

In [ ]:
# 3. We also expose a Clifford simulator with same API as the GPU and CPU simulators.

# Note that the Clifford simulator only accepts Clifford gates.
# So, we approximate the rz(1.86) rotation to an s gate.
qir_clifford = openqasm.compile("""
OPENQASM 3.0;
include "stdgates.inc";
qubit[4] q;
h q[0];
s q[0]; // approximation of rz(1.86)
h q[0];
sdg q[0];
barrier q;
x q[1];
cx q[0], q[1];
cx q[0], q[2];
cx q[1], q[3];
h q;
bit[4] c = measure q;
""")

results = run_qir_clifford(qir_clifford, shots=SHOTS, noise=noise)
bitstrings = [
    "".join("1" if bit == qsharp.Result.One else "0" for bit in measurement)
    for measurement in results
]
qsharp_widgets.Histogram(bitstrings, items = "top-25", sort="high-to-low")